In [1]:
from dotenv import load_dotenv
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_groq import ChatGroq
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools.structured import StructuredTool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool

import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from InvestingChatBot import InvestingChatBot

from langchain.chat_models import init_chat_model

In [2]:
REDIS_URL = "redis://localhost:6379/0"

In [3]:
load_dotenv()

True

In [4]:
def prompt_llm_tools(llm, tools: list, query: str):
    '''
    Prompts an LLM given a list of tools and a query
    '''
    llm_with_tools = llm.bind_tools(tools)
    messages = [HumanMessage(query)]
    ai_msg = llm_with_tools.invoke(query)

    messages.append(ai_msg)
    d = {tool.name: tool for tool in tools}

    for tool_call in ai_msg.tool_calls:
        selected_tool = d[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)

    response = llm_with_tools.invoke(messages)
    return response

In [5]:
# example usage with the Google Search API
# Define the search function
def topk_results(query):
    search = GoogleSearchAPIWrapper(k=10)  
    return search.results(query, 10)

# Create the tool
search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=topk_results,
)

# Initialize LLM from Groq
llm = ChatGroq(
    temperature=0,
    model='llama-3.3-70b-versatile'
)

tools = [search_tool]
prompt_llm_tools(llm, tools, "What is the weather today in Singapore?")

AIMessage(content='The current weather in Singapore is mostly cloudy with scattered thunderstorms and a high of 87F (30C) and a low of 77F (25C). There is a 70% chance of rain.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 1108, 'total_tokens': 1153, 'completion_time': 0.163636364, 'prompt_time': 0.04398784, 'queue_time': 0.26532203, 'total_time': 0.207624204}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run-b42f04f6-50bb-434a-b383-5f3e23e5ad00-0', usage_metadata={'input_tokens': 1108, 'output_tokens': 45, 'total_tokens': 1153})

In [6]:
# TODO: add more tools for calculations and integrate with LLM
# TODO: update the class file for the LLM to include the tools and agent?
# NOTE: agent is not needed maybe, can just use RAG or something like that, then we just scrape the web for financial data and put into vector store
# TODO: any way to do fundamental analysis on the stock we are interested in?

## Building the agent with the tools

In [7]:
# define the tools for the LLM
@tool
def magic_function(a: int, b: int) -> int:
    """Applies a magic function to an input."""
    return a + b * 10

@tool
def get_prices(symbol: str, date: str):
    '''
    Gets the prices of a stock symbol

    Args:
        symbol (str): The stock symbol
        date (str): The date of the stock price in ISO format

    Returns:
        float: The stock price given a particular day
    '''
    av = AlphaVantageAPIWrapper()
    prices = av._get_time_series_daily(symbol)
    prices = prices['Time Series (Daily)']
    return prices[date]['4. close']

@tool
def get_news_sentiment(symbol: str):
    '''
    Gets the sentiment of news articles for a stock symbol

    Args:
        symbol (str): The stock symbol

    Returns:
        float: The sentiment of news articles
    '''
    av = AlphaVantageAPIWrapper()
    news = av._get_market_news_sentiment(symbol)
    return news

@tool
def calculate_graham_number(earnings_per_share: float, book_value_per_share: float):
    '''
    Calculates the Graham Number

    Args:
        earnings_per_share (float): The earnings per share of the stock
        book_value_per_share (float): The book value per share of the stock

    Returns:
        float: The Graham Number
    '''
    return (22.5 * earnings_per_share * book_value_per_share) ** 0.5

@tool
def calculate_roe(net_income: float, shareholders_equity: float):
    '''
    Calculates the Return on Equity

    Args:
        net_income (float): The net income of the company
        shareholders_equity (float): The shareholders equity of the company

    Returns:
        float: The Return on Equity
    '''
    return net_income / shareholders_equity

In [8]:
def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

model = ChatGroq(
    temperature=0,
    model='llama-3.3-70b-versatile'
)
memory = RedisChatMessageHistory(session_id="foobar", url=REDIS_URL)

# Define the system prompt
system_prompt = """
You are an investing professional, specializing in stock-picking. You follow Warren Buffett's stock-picking style and strategies.
Answer the user's questions on investing in detail.
You may also be asked to perform fundamental analysis on a stock.

Given a chat history and the latest user question which might reference context in the chat history, answer the question
to the best of your ability. If you do not know the answer, just return "I don't know".
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        # First put the history
        ("placeholder", "{chat_history}"),
        # Then the new input
        ("human", "{input}"),
        # Finally the scratchpad
        ("placeholder", "{agent_scratchpad}"),
    ]
)
# dont use now since we hit the api limit for AlphaVantage
# tools = [magic_function, get_prices, get_news_sentiment]
tools = [calculate_graham_number, calculate_roe]


agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

config = {"configurable": {"session_id": "foobar"}}
print(
    agent_with_chat_history.invoke(
        {"input": "Hi, I'm Jia Hao! What's the ROE of a stock with net income of 1000 and shareholders equity of 1?"}, config
    )["output"]
)
print("---")
print(agent_with_chat_history.invoke({"input": "Remember my name?"}, config)["output"])
print("---")
print(
    agent_with_chat_history.invoke({"input": "what was that output again?"}, config)[
        "output"
    ]
)

The ROE of the stock is 1000.0. This means that for every dollar of shareholders' equity, the company generated $1000 in net income.
---
Your name is Jia Hao. The ROE of the stock is 1000.0. This means that for every dollar of shareholders' equity, the company generated $1000 in net income.
---
The ROE of the stock is 1000.0. This means that for every dollar of shareholders' equity, the company generated $1000 in net income.


In [9]:
history = get_message_history("foobar")
history.clear()

In [10]:
# example usage
model = ChatGroq(
    temperature=0,
    model='llama-3.3-70b-versatile'
)

tools = [calculate_graham_number, calculate_roe]

session_id = "foobar"

In [11]:
bot = InvestingChatBot(
    'llama-3.3-70b-versatile',
    0,
    tools,
    session_id
)

In [12]:
bot.clear_history()

In [14]:
bot.prompt("Hi, I'm Jia Hao! What's the ROE of a stock with net income of 1000 and shareholders equity of 1?")

"The Return on Equity (ROE) of the stock is 1000.0. This means that for every dollar of shareholders' equity, the company generated $1000 in net income."

In [15]:
bot.prompt("Remember my name?")

"Your name is Jia Hao. I remember you asked about the ROE of a stock with a net income of 1000 and shareholders' equity of 1. The ROE is 1000.0."